##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# String Unicode
<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/unicode"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Lihat di TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/load_data/unicode.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Jalankan di Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/load_data/unicode.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Liat source di GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/load_data/unicode.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Unduh notebook</a>
  </td>
</table>

## Pengenalan
Model yang memproses bahasa alami (*natural language*) sering menangani bahasa dan simbol/karakter yang berbeda-beda. *Unicode* adalah standar sistem pengkodean yang digunakan untuk mewakili simbol/karakter dari hampir semua bahasa. Setiap karakter dikodekan menggunakan integer unik [*code point*] (https://en.wikipedia.org/wiki/Code_point) antara `0` dan` 0x10FFFF`. Sebuah *Unicode string* adalah rangkaian dari nol atau lebih *code point*.

Tutorial ini, menunjukkan bagaimana cara merepresentasikan string Unicode di TensorFlow dan memanipulasinya menggunakan Unicode yang ekuivalen dengan standar operasi string. Cara ini, bertujuan untuk memisahkan *Unicode strings* menjadi beberapa token berdasarkan kata-kata yang dideteksi.

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version hanya ada di Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

## Tipe data `tf.string`

TensorFlow `tf.string`` dtype` memungkinkan Anda membangun tensor dari beberapa byte string.
*Unicode strings* secara default, menggunakan utf-8.

In [ ]:
tf.constant(u"Thanks 😊")

Tensor `tf.string` dapat menampung string byte dengan panjang yang berbeda-beda karena string byte diperlakukan sebagai unit atom. Panjang string tidak masuk dalam dimensi tensor.

In [ ]:
tf.constant([u"You're", u"welcome!"]).shape

Catatan: Saat menggunakan python untuk membangun string, penanganan unicode berbeda antara v2 dan v3. Dalam v2, *unicode strings* ditunjukkan oleh awalan "u", seperti di atas. Di v3, strings adalah unicode yang terkodekan secara default.

## Mewakili Unicode

Ada dua cara standar untuk menunjukan string Unicode di TensorFlow:

* `string` scalar - di mana urutan code points dikodekan menggunakan  [character encoding](https://en.wikipedia.org/wiki/Character_encoding).
* `int32` vector - di mana setiap posisi berisi satu code points.

Misalnya, tiga nilai berikut semuanya mewakili Unicode string `"语言处理"` (yang berarti "pemrosesan bahasa" dalam bahasa Cina):

In [ ]:
# Unicode string, ditunjukan sebagai string scalar yang dikodekan menggunakan format UTF-8
text_utf8 = tf.constant(u"语言处理")
text_utf8

In [ ]:
# Unicode string, ditunjukan sebagai string scalar yang dikodekan menggunakan format UTF-16-BE
text_utf16be = tf.constant(u"语言处理".encode("UTF-16-BE"))
text_utf16be

In [ ]:
# Unicode string, ditunjukan sebagai sebuah vektor dalam code point Unicode.
text_chars = tf.constant([ord(char) for char in u"语言处理"])
text_chars

### Konversi di antara representasi

TensorFlow menyediakan operasi untuk mengkonversi vektor, skalar, atau format :

* `tf.strings.unicode_decode`: Mengonversi skalar string yang dikodekan ke vektor.
* `tf.strings.unicode_encode`: Mengonversi vektor ke skalar string yang dikodekan.
* `tf.strings.unicode_transcode`: Mengonversi skalar string yang dikodekan ke format pengkodean yang berbeda.

In [ ]:
tf.strings.unicode_decode(text_utf8,
                          input_encoding='UTF-8')

In [ ]:
tf.strings.unicode_encode(text_chars,
                          output_encoding='UTF-8')

In [ ]:
tf.strings.unicode_transcode(text_utf8,
                             input_encoding='UTF8',
                             output_encoding='UTF-16-BE')

### Dimensi batch

Saat mendekode beberapa string, jumlah karakter dalam setiap string mungkin tidak sama. Hasil returnnya adalah [`tf.RaggedTensor`](../../guide/ragged_tensor.ipynb), di mana panjang dimensi bervariasi ,tergantung pada jumlah karakter di setiap string:

In [ ]:
# A batch of Unicode strings, each represented as a UTF8-encoded string.
batch_utf8 = [s.encode('UTF-8') for s in
              [u'hÃllo',  u'What is the weather tomorrow',  u'Göödnight', u'😊']]
batch_chars_ragged = tf.strings.unicode_decode(batch_utf8,
                                               input_encoding='UTF-8')
for sentence_chars in batch_chars_ragged.to_list():
  print(sentence_chars)

Anda dapat menggunakan `tf.RaggedTensor` ini secara langsung, atau mengonversinya menjadi` tf.Tensor` dengan padding atau `tf.SparseTensor` menggunakan metode` tf.RaggedTensor.to_tensor` dan `tf.RaggedTensor.to_sparse`.

In [ ]:
batch_chars_padded = batch_chars_ragged.to_tensor(default_value=-1)
print(batch_chars_padded.numpy())

In [ ]:
batch_chars_sparse = batch_chars_ragged.to_sparse()

Saat mengkodekan beberapa string dengan panjang yang sama, `tf.Tensor` dapat digunakan sebagai input:

In [ ]:
tf.strings.unicode_encode([[99, 97, 116], [100, 111, 103], [ 99, 111, 119]],
                          output_encoding='UTF-8')

Saat mengkodekan beberapa string dengan panjang yang bervariasi, `tf.RaggedTensor` dapat digunakan sebagai input:

In [ ]:
tf.strings.unicode_encode(batch_chars_ragged, output_encoding='UTF-8')

Jika Anda memiliki tensor dengan beberapa string dalam format padded atau sparse, konversikan ke `tf.RaggedTensor` sebelum memanggil` unicode_encode`:

In [ ]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_sparse(batch_chars_sparse),
    output_encoding='UTF-8')

In [ ]:
tf.strings.unicode_encode(
    tf.RaggedTensor.from_tensor(batch_chars_padded, padding=-1),
    output_encoding='UTF-8')

## Operasi Unicode 

### Panjang karakter

Operasi `tf.strings.length` memiliki parameter` unit`, yang menunjukkan cara/format karakter harus dikodekan. `unit` secara default menjadi` "BYTE" `, tetapi dapat disetel ke nilai lain, seperti` "UTF8_CHAR" `atau` "UTF16_CHAR" `, untuk menentukan jumlah code point Unicode di setiap` string` yang dikodekan.

In [ ]:
# Perhatikan bahwa karakter terakhir membutuhkan 4 byte dalam UTF8.
thanks = u'Thanks 😊'.encode('UTF-8')
num_bytes = tf.strings.length(thanks).numpy()
num_chars = tf.strings.length(thanks, unit='UTF8_CHAR').numpy()
print('{} bytes; {} UTF-8 characters'.format(num_bytes, num_chars))

### Substring karakter

Hampir sama, operasi `tf.strings.substr` terdapat parameter" `unit`", dan digunakan untuk menentukan jenis offset apa yang ada dalam parameter "` pos` "dan" `len`".

In [ ]:
# default: unit='BYTE'. With len=1, we return a single byte.
# default: unit='BYTE'. Dengan len=1, menunjukan 1  byte.
tf.strings.substr(thanks, pos=7, len=1).numpy()

In [ ]:
# Specifying unit='UTF8_CHAR', we return a single character, which in this case
# is 4 bytes.
# Dengan mengatur unit='UTF8_CHAR', kita menunjukan 1 karakter, dengan panjang
# 4 bytes.
print(tf.strings.substr(thanks, pos=7, len=1, unit='UTF8_CHAR').numpy())

### Memecah Unicode Strings

Operasi `tf.strings.unicode_split` memecah string unicode menjadi substring karakter:

In [ ]:
tf.strings.unicode_split(thanks, 'UTF-8').numpy()

### Byte offset untuk karakter

Untuk menyelaraskan tensor karakter yang dihasilkan oleh `tf.strings.unicode_decode` dengan string asli, penting untuk mengetahui offset pada setiap awal karakter. Metode `tf.strings.unicode_decode_with_offsets` mirip dengan` unicode_decode`, tapi metode ini me-return tensor kedua yang berisi offset awal setiap karakter.

In [ ]:
codepoints, offsets = tf.strings.unicode_decode_with_offsets(u"🎈🎉🎊", 'UTF-8')

for (codepoint, offset) in zip(codepoints.numpy(), offsets.numpy()):
  print("At byte offset {}: codepoint {}".format(offset, codepoint))

## Script Unicode

Setiap code point Unicode, merupakan milik satu kumpulan codepoint yang dikenal sebagai [script](https://en.wikipedia.org/wiki/Script_%28Unicode%29). Script dari sebuah karakter sangat membantu dalam menentukan bahasa yang mungkin digunakan karakter tersebut. Misalnya, mengetahui bahwa 'Б' dalam script Cyrillic menunjukkan bahwa teks modern yang mengandung karakter tersebut kemungkinan berasal dari bahasa Slavik seperti Rusia atau Ukraina.

TensorFlow menyediakan operasi `tf.strings.unicode_script` untuk menentukan script mana yang digunakan oleh code point tertentu. Script code yang bernilai `int32` sesuai dengan nilai [International Components for
Unicode](http://site.icu-project.org/home) (ICU) [`UScriptCode`](http://icu-project.org/apiref/icu4c/uscript_8h.html).

In [ ]:
uscript = tf.strings.unicode_script([33464, 1041])  # ['芸', 'Б']

print(uscript.numpy())  # [17, 8] == [USCRIPT_HAN, USCRIPT_CYRILLIC]

The `tf.strings.unicode_script` operation can also be applied to multidimensional `tf.Tensor`s or `tf.RaggedTensor`s of codepoints:

Operasi `tf.strings.unicode_script` juga dapat diterapkan ke` tf.Tensor` atau `tf.RaggedTensor` dari codepoint:

In [ ]:
print(tf.strings.unicode_script(batch_chars_ragged))

## Contoh: Segmentasi sederhana

Segmentasi adalah memisahkan teks menjadi satuan kata-kata. Segmentasi mudah dilakukan ketika karakter spasi digunakan untuk memisahkan antar kata, tetapi beberapa bahasa (seperti Cina dan Jepang) tidak menggunakan spasi, dan beberapa bahasa (seperti Jerman) mengandung gabungan yang harus dipecah untuk menganalisis artinya. Dalam teks web, berbagai bahasa dan script sering dicampur bersama, seperti dalam "NY 株 価" (Bursa Saham New York).

Kita dapat melakukan segmentasi yang sangat kasar (tanpa menerapkan model ML) dengan menggunakan perubahan nilai pada script untuk memperkirakan pembatas kata. Cara ini dapat digunakan untuk string seperti contoh "NY 株 価" di atas. Cara ini juga dapat digunakan untuk sebagian besar bahasa yang menggunakan spasi, karena karakter spasi dari berbagai script semuanya diklasifikasikan sebagai USCRIPT_COMMON, kode skrip khusus yang berbeda dengan teks aktual apa pun.

In [ ]:
# dtype: string; shape: [num_sentences]
#
# sebuah kalimat untuk contoh.  Silakan diganti menggunakan bahasa lain untuk mencoba input yang berbeda!
sentence_texts = [u'Hello, world.', u'世界こんにちは']

First, we decode the sentences into character codepoints, and find the script identifeir for each character.

Pertama, kita menkodedan kalimat menjadi codepoints karakter, dan menemukan pengidentifikasi script untuk setiap karakter.

In [ ]:
# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_codepoint[i, j] adalah codepoint untuk karakter urutan ke j pada
# kalimat urutan ke i.
sentence_char_codepoint = tf.strings.unicode_decode(sentence_texts, 'UTF-8')
print(sentence_char_codepoint)

# dtype: int32; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_scripts[i, j] adalah unicode script untuk karakter urutan ke j pada
# kalimat urutan ke i.
sentence_char_script = tf.strings.unicode_script(sentence_char_codepoint)
print(sentence_char_script)

Selanjutnya, kita menggunakan pengidentifikasi script tersebut untuk menentukan di mana batas katanya. Kita menambahkan batas kata di awal setiap kalimat, dan untuk setiap karakter yang scripnya berbeda dari karakter sebelumnya:

In [ ]:
# dtype: bool; shape: [num_sentences, (num_chars_per_sentence)]
#
# sentence_char_starts_word[i, j] bernilai True jika karakter urutan ke j dalam
# kalimat urutan ke i adalah awal dari sebuah kata.
sentence_char_starts_word = tf.concat(
    [tf.fill([sentence_char_script.nrows(), 1], True),
     tf.not_equal(sentence_char_script[:, 1:], sentence_char_script[:, :-1])],
    axis=1)

# dtype: int64; shape: [num_words]
#
# word_starts[i] adalah index dari karakter yang merupakan awal dari urutan kata ke i (dalam
# daftar kata dari kalimat).
word_starts = tf.squeeze(tf.where(sentence_char_starts_word.values), axis=1)
print(word_starts)

Kita kemudian dapat menggunakan *start offsets* tersebut untuk membangun sebuah `RaggedTensor` yang terdiri atas list kata-kata dari seluruh batch:

In [ ]:
# dtype: int32; shape: [num_words, (num_chars_per_word)]
#
# word_char_codepoint[i, j] adalah codepoint untuk urutan karakter ke j dalam
# urutan kata ke i.
word_char_codepoint = tf.RaggedTensor.from_row_starts(
    values=sentence_char_codepoint.values,
    row_starts=word_starts)
print(word_char_codepoint)

Dan akhirnya, kita dapat mengelompokkan codepoint kata `RaggedTensor` kembali ke kalimat:

In [ ]:
# dtype: int64; shape: [num_sentences]
#
# sentence_num_words[i] adalah jumlah kata dalam urutan kalimat ke-i.
sentence_num_words = tf.reduce_sum(
    tf.cast(sentence_char_starts_word, tf.int64),
    axis=1)

# dtype: int32; shape: [num_sentences, (num_words_per_sentence), (num_chars_per_word)]
#
# sentence_word_char_codepoint[i, j, k] adalah codepoint untuk urutan karakter ke k
# dalam urutan kata ke j dalam urutan kaliman ke i.
sentence_word_char_codepoint = tf.RaggedTensor.from_row_lengths(
    values=word_char_codepoint,
    row_lengths=sentence_num_words)
print(sentence_word_char_codepoint)

Untuk membuat hasil akhir lebih mudah dibaca, kita dapat mengkodekannnya kembali ke string UTF-8:

In [ ]:
tf.strings.unicode_encode(sentence_word_char_codepoint, 'UTF-8').to_list()